# Tutorial 3: Hyperparameter Tuning with Optuna

This tutorial shows how to run Optuna sweeps inside a ZenML pipeline. You'll learn how to:

- Generate synthetic graph data for tuning
- Define a ZenML pipeline that includes an Optuna search step
- Inspect the best hyperparameters and the tuned model
- Evaluate the tuned model on the validation split

Unlike earlier versions, the full pipeline now lives in the notebook so you can modify
every step. We assume Optuna is installed (see `requirements.txt`).


In [1]:
import contextlib
import io
import torch
import optuna
import pytorch_lightning as pl
from torch_geometric.data import Data
from zenml import pipeline, step

from pioneerml.models import GroupClassifier
from pioneerml.training import GraphDataModule, GraphLightningModule
from pioneerml.zenml.materializers import (
    GraphDataModuleMaterializer,
    PyGDataListMaterializer,
)
from pioneerml.zenml import load_step_output
from pioneerml.zenml import utils as zenml_utils
from pioneerml.zenml.utils import detect_available_accelerator

zenml_client = zenml_utils.setup_zenml_for_notebook(use_in_memory=True)
print(f"ZenML initialized with stack: {zenml_client.active_stack_model.name}")


ZenML initialized with stack: default


## Build the Tuning Pipeline

The pipeline mirrors the structure from earlier tutorials and adds an Optuna step:

1. `create_data`: synthetic graph classification dataset
2. `create_datamodule`: wraps the dataset with train/val splits
3. `run_hyperparameter_search`: Optuna study that picks hidden size, learning rate, dropout
4. `train_best_model`: trains one final model with the best parameters

Optuna runs entirely inside the pipeline, making the sweep reproducible and tracked in ZenML.


In [2]:

def create_synthetic_tuning_data(num_samples: int = 400) -> list[Data]:
    """Generate a deliberately tricky dataset so Optuna can't hit 100% accuracy."""
    data: list[Data] = []

    class_means = torch.tensor([
        [0.7, -0.3, 0.2, 0.1, -0.1],
        [-0.1, 0.8, -0.2, 0.3, 0.15],
        [-0.6, -0.4, 0.4, -0.2, 0.25],
    ])
    class_drift = torch.tensor([
        [0.4, 0.3, -0.1, 0.0, 0.2],
        [-0.3, 0.2, 0.25, -0.15, -0.1],
        [0.2, -0.4, 0.15, 0.25, -0.05],
    ])
    feature_scales = torch.tensor([
        [1.0, 0.9, 1.1, 0.95, 1.05],
        [0.95, 1.05, 0.85, 1.1, 0.9],
        [1.1, 0.95, 0.9, 1.0, 1.0],
    ])

    for _ in range(num_samples):
        label = torch.randint(0, 3, (1,)).item()
        num_nodes = torch.randint(7, 18, (1,)).item()

        mix_label = (label + torch.randint(1, 3, (1,)).item()) % 3
        mix_ratio = torch.rand(1).item() * 0.6 + 0.2
        prototype = mix_ratio * class_means[label] + (1 - mix_ratio) * class_means[mix_label]

        t = torch.linspace(0, 1, steps=num_nodes).unsqueeze(1)
        wiggles = torch.cat([
            torch.sin(2.0 * torch.pi * t + label * 0.3),
            torch.cos(3.0 * torch.pi * t + mix_ratio),
            torch.sin(4.0 * torch.pi * t - mix_ratio * 0.5),
            torch.cos(5.0 * torch.pi * t + label * 0.2),
            torch.sin(6.0 * torch.pi * t - 0.1),
        ], dim=1)

        x = prototype + 0.4 * wiggles
        x = x * feature_scales[label]

        noise = torch.randn(num_nodes, 5)
        correlated = noise + 0.35 * torch.matmul(noise, torch.ones(5, 5) * 0.1)
        drift = class_drift[label] * torch.randn(num_nodes, 1)
        x = x + 0.25 * correlated + drift

        projection = torch.randn(5, 5) * 0.2 + torch.eye(5)
        x = x @ projection

        if label == 0:
            ring = torch.stack([
                torch.arange(num_nodes),
                (torch.arange(num_nodes) + 1) % num_nodes,
            ])
            random_edges = torch.randint(0, num_nodes, (2, num_nodes * 2))
            edge_index = torch.cat([ring, random_edges], dim=1)
        elif label == 1:
            cluster = max(3, num_nodes // 2)
            src = torch.randint(0, cluster, (num_nodes * 3,))
            dst = torch.randint(0, cluster, (num_nodes * 3,))
            long_jump = torch.randint(0, num_nodes, (2, num_nodes))
            edge_index = torch.cat([torch.stack([src, dst], dim=0), long_jump], dim=1)
        else:
            src = torch.randint(0, num_nodes, (num_nodes * 2,))
            dst = (src + torch.randint(2, 7, (num_nodes * 2,))) % num_nodes
            extra = torch.randint(0, num_nodes, (2, num_nodes))
            edge_index = torch.cat([torch.stack([src, dst], dim=0), extra], dim=1)

        edge_attr = torch.randn(edge_index.shape[1], 4) * 0.15 + label * 0.04

        noisy_label = label
        if torch.rand(1).item() < 0.1:
            noisy_label = torch.randint(0, 3, (1,)).item()

        y = torch.zeros(3)
        y[noisy_label] = 1.0

        data.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))

    return data


def _run_quietly(fn):
    """Suppress Lightning/stdout noise while preserving Optuna logs."""
    buffer_out = io.StringIO()
    buffer_err = io.StringIO()
    with contextlib.redirect_stdout(buffer_out), contextlib.redirect_stderr(buffer_err):
        return fn()


@step(output_materializers=PyGDataListMaterializer, enable_cache=False)
def create_data() -> list[Data]:
    """Step 1: Generate synthetic graphs for tuning."""
    return create_synthetic_tuning_data()


@step(output_materializers=GraphDataModuleMaterializer, enable_cache=False)
def create_datamodule(data: list[Data]) -> GraphDataModule:
    """Step 2: Wrap the dataset in a Lightning DataModule."""
    return GraphDataModule(dataset=data, val_split=0.25, batch_size=32, num_workers=0)


@step(enable_cache=False)
def run_hyperparameter_search(datamodule: GraphDataModule, n_trials: int = 4) -> dict:
    """Step 3: Perform an Optuna search over hidden size, dropout, and learning rate."""

    def objective(trial: optuna.Trial) -> float:
        hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
        dropout = trial.suggest_float("dropout", 0.0, 0.3)
        lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)

        model = GroupClassifier(num_classes=3, hidden=hidden_dim, dropout=dropout)
        lightning_module = GraphLightningModule(model, task="classification", lr=lr)
        accelerator, devices = detect_available_accelerator()

        trainer = pl.Trainer(
            accelerator=accelerator,
            devices=devices,
            max_epochs=2,
            logger=False,
            enable_checkpointing=False,
            enable_progress_bar=False,
        )

        datamodule.setup(stage="fit")

        def fit():
            trainer.fit(lightning_module, datamodule=datamodule)

        def validate():
            return trainer.validate(lightning_module, datamodule=datamodule, verbose=False)

        _run_quietly(fit)
        val_metrics = _run_quietly(validate)

        if val_metrics and isinstance(val_metrics[0], dict):
            accuracy = val_metrics[0].get("val_accuracy")
            if accuracy is not None:
                return float(accuracy)
            loss = val_metrics[0].get("val_loss")
            if loss is not None:
                return 1.0 / (1.0 + float(loss))
        return 0.0

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    return {
        "best_hidden_dim": study.best_params["hidden_dim"],
        "best_dropout": study.best_params["dropout"],
        "best_lr": study.best_params["lr"],
        "best_accuracy": study.best_value,
        "n_trials": len(study.trials),
    }


@step(enable_cache=False)
def train_best_model(best_params: dict, datamodule: GraphDataModule) -> GraphLightningModule:
    """Step 4: Train a final model with the best Optuna parameters."""
    model = GroupClassifier(
        num_classes=3,
        hidden=best_params["best_hidden_dim"],
        dropout=best_params["best_dropout"],
    )
    lightning_module = GraphLightningModule(
        model,
        task="classification",
        lr=best_params["best_lr"],
    )

    accelerator, devices = detect_available_accelerator()
    trainer = pl.Trainer(
        accelerator=accelerator,
        devices=devices,
        max_epochs=5,
        logger=False,
        enable_checkpointing=False,
        enable_progress_bar=False,
    )

    datamodule.setup(stage="fit")

    def fit():
        trainer.fit(lightning_module, datamodule=datamodule)

    _run_quietly(fit)
    return lightning_module.eval()


@pipeline
def tuning_pipeline(n_trials: int = 4):
    data = create_data()
    datamodule = create_datamodule(data)
    best_params = run_hyperparameter_search(datamodule, n_trials=n_trials)
    tuned_model = train_best_model(best_params, datamodule)
    return tuned_model, datamodule, best_params



## Run the Optuna Sweep

Execute the pipeline with a small number of trials (increase `n_trials` for real sweeps).
The pipeline stores all runs and best parameters in ZenML so you can reproduce the sweep later.


In [3]:
run = tuning_pipeline.with_options(enable_cache=False)(n_trials=4)
print(f"Pipeline run status: {run.status}")

tuned_module = load_step_output(run, "train_best_model")
datamodule = load_step_output(run, "create_datamodule")
best_params = load_step_output(run, "run_hyperparameter_search")

if tuned_module is None or datamodule is None or best_params is None:
    raise RuntimeError("Failed to load artifacts from the tuning pipeline.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tuned_module = tuned_module.to(device).eval()
datamodule.setup(stage="fit")

print("Best hyperparameters:")
for key, value in best_params.items():
    print(f"- {key}: {value}")


Initiating a new run for the pipeline: tuning_pipeline.
Caching is disabled by default for tuning_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
  deployer: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step create_data has started.
Step create_data has finished in 0.117s.
Step create_datamodule has started.
Step create_datamodule has finished in 0.065s.
Step run_hyperparameter_search has started.


[I 2025-11-25 04:12:55,129] A new study created in memory with name: no-name-325d2f6e-ce2d-4d3b-8a5c-4fba5c15cd97
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | GroupClassifier   | 118 K  | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
118 K     Trainable params
0         Non-trainable params
118 K     Total params
0.472     Total estimated model params size (MB)
47       

[run_hyperparameter_search] /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the num_workers argument to num_workers=15 in the DataLoader` to improve performance.



[W 2025-11-25 04:12:55,356] Trial 0 failed with parameters: {'hidden_dim': 64, 'dropout': 0.08534222827635905, 'lr': 0.00010689969563242122} because of the following error: RuntimeError('mat1 and mat2 shapes cannot be multiplied (326x6 and 5x64)').
Traceback (most recent call last):
[run_hyperparameter_search] Traceback (most recent call last):
  File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
[run_hyperparameter_search]   File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_78200/663702664.py", line 128, in objective
    trainer.fit(lightning_module, datamodule=datamodule)
[run_hyperparameter_search]   File "/tmp/ipykernel_78200/663702664.py",

Failed to run step run_hyperparameter_search: mat1 and mat2 shapes cannot be multiplied (326x6 and 5x64)
Step run_hyperparameter_search failed.
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/local/local_orchestrator.py", line 160, in submit_pipeline
    self.run_step(step=step)
  File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/local/local_orchestrator.py", line 160, in submit_pipeline
    self.run_step(step=step)
  File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/base_orchestrator.py", line 445, in run_step
    launch_step(
  File "/home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/base_orchestrator.py", line 445, in run_step
    launch_step(
  File "/home/jack/virtual

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 run = tuning_pipeline.with_options(enable_cache=False)(n_trials=4)                          │
│    2 print(f"Pipeline run status: {run.status}")                                                 │
│    3                                                                                             │
│    4 tuned_module = load_step_output(run, "train_best_model")                                    │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/pip │
│ elines/pipeline_definition.py:1627 in __call__                                                   │
│                                                                                                  │
│   1624 │   │   │   return self.entrypoint(*args, **kwargs)  # type: ignore[no-any-return]        │
│   1625 │   │                                                                                     │
│   1626 │   │   self.prepare(*args, **kwargs)                                                     │
│ ❱ 1627 │   │   return self._run()                                                                │
│   1628 │                                                                                         │
│   1629 │   def _call_entrypoint(self, *args: Any, **kwargs: Any) -> Any:                         │
│   1630 │   │   """Calls the pipeline entrypoint function with the given arguments.               │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/pip │
│ elines/pipeline_definition.py:1102 in _run                                                       │
│                                                                                                  │
│   1099 │   │   │   │   │   │   │   "`zenml login --local`."                                      │
│   1100 │   │   │   │   │   │   )                                                                 │
│   1101 │   │   │   │                                                                             │
│ ❱ 1102 │   │   │   │   submit_pipeline(                                                          │
│   1103 │   │   │   │   │   snapshot=snapshot, stack=stack, placeholder_run=run                   │
│   1104 │   │   │   │   )                                                                         │
│   1105                                                                                           │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/exe │
│ cution/pipeline/utils.py:115 in submit_pipeline                                                  │
│                                                                                                  │
│   112 │   │   │   │   # the run as failed if it's still in an unfinished state.                  │
│   113 │   │   │   │   publish_failed_pipeline_run(placeholder_run.id)                            │
│   114 │   │   │                                                                                  │
│ ❱ 115 │   │   │   raise e                                                                        │
│   116                                                                                            │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/l

## Inspect the Tuned Model

Check that the tuned model has the expected shape, parameter count, and device placement.


In [ ]:
device = next(tuned_module.parameters()).device
param_count = sum(p.numel() for p in tuned_module.parameters())

train_loader = datamodule.train_dataloader()
first_batch = next(iter(train_loader))

print("Tuned Model Summary:")
print(f"- Run: {run.name}")
print(f"- Device: {device}")
print(f"- Parameters: {param_count:,}")
print(f"- Nodes per batch: {first_batch.x.shape[0]} | Features: {first_batch.x.shape[1]}")
print(f"- Edges: {first_batch.edge_index.shape[1]}")

with torch.no_grad():
    logits = tuned_module(first_batch.to(device))
    print(f"- Output logits shape: {tuple(logits.shape)}")


## Evaluate Validation Accuracy

Run the tuned model on the validation split. We handle label shapes explicitly because
PyG batches graph-level labels into `(batch_size, num_classes)` tensors.


In [ ]:
val_loader = datamodule.val_dataloader()
if isinstance(val_loader, list) and len(val_loader) == 0:
    val_loader = datamodule.train_dataloader()

correct = 0
total = 0
tuned_module.eval()

for batch in val_loader:
    batch = batch.to(device)
    with torch.no_grad():
        logits = tuned_module(batch)

    labels = batch.y
    if labels.dim() == 1:
        if labels.shape[0] % 3 == 0:
            labels = labels.view(-1, 3)
        else:
            labels = labels.unsqueeze(0)
    labels = torch.argmax(labels, dim=1)

    preds = torch.argmax(logits, dim=1)
    correct += int((preds == labels).sum().item())
    total += int(labels.numel())

accuracy = correct / total if total > 0 else 0.0
print(f"Validation accuracy: {accuracy:.1%} ({correct}/{total})")
